<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
# Se instalan las librerías.
!pip install langchain==0.3.21
!pip install langchain-community==0.3.20
!pip install beautifulsoup4==4.13.3
!pip install langgraph==0.3.31
!pip install duckduckgo-search==8.0.0
!pip install tiktoken==0.9.0
!pip install pypdf==5.4.0
!pip install langchain-huggingface==0.1.2
!pip install langchain_deepseek==0.1.2

In [ ]:
# Se importan las librerías.
import os
import warnings
import sys
from typing import Any
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.vectorstores.base import VectorStore
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_deepseek import ChatDeepSeek
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import BaseChatMessageHistory, BaseMessage
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
import bs4
import pandas as pd
from uuid import uuid4

# Filtro para advertencias.
warnings.filterwarnings('ignore')

<br>

# Añadiendo un RAG como herramienta a un agente

In [ ]:
# Se carga la información desde distintas fuentes
web_loader = WebBaseLoader(web_paths=["https://dialektico.com/cama-ultra-lujosa-para-gatos-dialektiroyal-comfort/"])
documents = []

for doc in os.listdir('documents'):
  pdf_loader = PyPDFLoader('documents/'+ doc)
  async for page in pdf_loader.alazy_load():
      documents.append(page)

async for doc in web_loader.alazy_load():
    documents.append(doc)

In [ ]:
# Cantidad de documentos.
len(documents)

In [ ]:
# Se prepara el algoritmo de embeddings a utilizar.
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
# Se crea vector store en memoria para almacenar vectores.
vector_store = InMemoryVectorStore(embedding=embeddings_model)

# Se instancia el CharacterTextSplitter.
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=1000, chunk_overlap=200
)

# Se divide el texto de los documentos utilizando el tokenizador tiktoken.
chunks = text_splitter.split_documents(documents)

# Se almacenan los vectores.
vector_store.add_documents(documents=chunks, ids=[str(uuid4()) for _ in range(len(chunks))])

In [ ]:
# Se traen documentos con mayor similaridad.
question = "¿Cuál es el precio del suéter térmico de felpa para perros?"
retrieved_docs = vector_store.similarity_search(question, k=5)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [ ]:
docs_content

In [ ]:
def buscar_producto_en_vectorstore(prompt: str) -> str:
    """
    Realiza una búsqueda semántica en una base de datos vectorial para recuperar
    documentos relacionados con un producto, basándose en el contenido del prompt.

    El objetivo es obtener información útil sobre productos específicos
    (como características, materiales, tallas, precio, etc.) almacenada en una
    base vectorial de documentos.

    Args:
        * prompt (str): La consulta o pregunta sobre un producto, como "¿Qué
        características tienen la camiseta de lino para gatos y la
        camisa transpirable de mallas para perros?".

    Returns:
        str: contenidos (`page_content`) de los documentos
        más relevantes encontrados.

    Example:
        >>> buscar_producto_en_vectorstore(
                "¿Cuáles son las características ¿Qué características tienen la
                camiseta de lino para gatos y la camisa transpirable de mallas
                para perros??"
            )
        'Descripción: Suéter grueso y suave...', 'Colores: Gris claro, Vino...'
    """
    # Realizar la búsqueda semántica
    retrieved_docs = vector_store.similarity_search(prompt, k=5)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # Extraer y retornar el contenido de los documentos
    return docs_content



In [ ]:
prompt = "¿Cuál es el precio del suéter térmico de felpa para perros?"

buscar_producto_en_vectorstore(prompt)

In [ ]:
# Definimos la herramienta del buscador web.
search = DuckDuckGoSearchResults(max_results=5, description="""Herramienta que
puede ser utilizada para buscar información en internet, se puede buscar el
clima del día actual, por ejemplo: ¿cuál es la temperatura en Guadalajara
Jalisco el día de hoy?""")

def obtener_prendas_para_mascota(ruta_csv: str, animal: str, temperatura_aprox: float) -> list[str]:
    """
    Filtra una tabla CSV de prendas para mascotas y devuelve aquellas adecuadas
    según el tipo de animal (perro o gato) y la temperatura actual.

    La función clasifica la temperatura en una de tres categorías:
        - 'frío': temperatura menor a 15°C
        - 'templado': temperatura entre 15°C y 25°C (inclusive)
        - 'caluroso': temperatura mayor a 25°C

    Luego, filtra las prendas que coincidan con la clasificación de temperatura y el animal especificado.

    Args:
        ruta_csv (str): Ruta al archivo CSV que contiene la tabla de prendas.
        animal (str): Tipo de mascota, solo puede ser: 'gato' o 'perro'.
        temperatura_actual (float): Temperatura en grados Celsius.

    Returns:
        list[str]: Lista de nombres de prendas recomendadas para el animal y clima dados.

    Raises:
        FileNotFoundError: Si no se encuentra el archivo CSV en la ruta especificada.
        ValueError: Si el tipo de animal no está presente en la tabla.

    Example:
        >>> obtener_prendas_para_mascota('prendas_mascotas.csv', 'perro', 10)
        ['chamarra acolchada impermeable', 'suéter térmico de felpa']
    """
    # Cargar la tabla
    df = pd.read_csv(ruta_csv)

    # Clasificar la temperatura
    if temperatura_aprox < 15:
        clasificacion_temp = 'frío'
    elif 15 <= temperatura_aprox <= 25:
        clasificacion_temp = 'templado'
    else:
        clasificacion_temp = 'caluroso'

    # Verificar si el animal está en el dataset
    if animal.lower() not in df['animal'].str.lower().unique():
        raise ValueError(f"Animal '{animal}' no encontrado en la tabla.")

    # Filtrar según clasificación de temperatura y animal
    prendas_filtradas = df[
        (df['temperatura'] == clasificacion_temp) &
        (df['animal'].str.lower() == animal.lower())
    ]

    return prendas_filtradas['nombre_prenda'].tolist()

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')

# Se define el modelo a utilizar y añaden valores de parámetros.
model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=300
      )

# Creando multiRAG agéntico con LangGraph

Ahora utilizaremos `create_react_agent` para crear un agente que utilice estas herramientaS.

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/04/graph_lc.png" width="200" /></center>



Para crear un multiRag agéntico añadiremos una nueva herramienta a la lista y recrearemos el agente.

Lo haremos para crear un agente que atienda a una tienda electrónica de accesorios para mascotas.

El agente debe hacer lo siguiente:



*   Investigar la temperatura de la ciudad donde vive el cliente.
*   Conociendo la temperatura, y el tipo de mascota del cliente (perro o gato), extraer información de los productos que puede ofrecer de un CSV.
* Dar la información al cliente.

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/04/Diagrama_agente_2.drawio.png" width="1000" /></center>


In [ ]:
# Se crea el agente con herramientas nuevas.
tools = [search, obtener_prendas_para_mascota, buscar_producto_en_vectorstore]
agent_executor = create_react_agent(model, tools)

# Se añaden instrucciones.
system_message = """Eres un asistente de una tienda en línea que vende productos
        para mascotas. El cliente te dirá qué mascota tiene (perro o gato) y en
        qué ciudad vivo, y tú debes de seguir los siguientes pasos:
        - Debes de utilizar la herramienta search para obtener la temperatura
        aproximada de la ciudad donde vive, colocando en el buscador:
        cuál es el clima actual en {ciudad del cliente}.
        - Utilizando la información del buscador, utilizarás la herramienta
        obtener_prendas_para_mascota, añadiendo los argumentos correspondientes,
        donde el csv con la información está en /content/productos.csv, y añades
        tipo de animal y temperatura aproximada para obtener los datos.
        - Utilizando estos datos, utiliza la herramienta buscar_producto_en_vectorstore,
        añade como argumento un prompt que solicite información de los productos
        que obtuviste con la herramienta obtener_prendas_para_mascota.
        - Finalmente, utiliza la información que obtuviste de la herramienta
        buscar_producto_en_vectorstore para dar la información al cliente de los
        productos que le puedes ofrecer, dile los nombres de los productos, su
        descripíción, colores, tallas y precio, no olvides ser amable.
        Utiliza pocas oraciones y mantén la respuesta lo más concisa posible.
        """


In [ ]:
# Se utiliza el agente haciendo stream en la respuesta.
for step, metadata in agent_executor.stream(
    {"messages": [SystemMessage(
        content=system_message),
                  HumanMessage(content="¿Hola, vivo en Manchester, Inglaterra, qué ropa tienes para gato?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from langchain.schema import SystemMessage, HumanMessage
from uuid import uuid4
from typing import Any

def agentic_multirag(agent_executor: Any, system_message: str) -> None:
    session_id = str(uuid4())
    config = {"configurable": {"thread_id": session_id}}

    textarea = widgets.Textarea(
        value='',
        placeholder='Escribe tu pregunta aquí...',
        description='Pregunta:',
        layout=widgets.Layout(width='100%', height='100px')
    )

    enviar_button = widgets.Button(description="Enviar", button_style='primary')
    output = widgets.Output()

    def on_submit(_):
        user_question = textarea.value.strip()
        textarea.value = ''
        output.clear_output()

        if user_question.lower() in ('salir', 'exit', 'quit'):
            with output:
                display(Markdown("🚪 **Saliendo del chat. ¡Hasta luego!**"))
            return

        full_response = ""

        with output:
            try:
                for step, metadata in agent_executor.stream(
                    {
                        "messages": [
                            SystemMessage(content=system_message),
                            HumanMessage(content=user_question)
                        ]
                    },
                    config,
                    stream_mode="messages",
                ):
                    if metadata["langgraph_node"] == "agent" and (text := step.text()):
                        full_response += text

                # Mostrar la respuesta completa como Markdown
                display(Markdown(full_response))
                display(Markdown("\n📝 *Escribe otra pregunta (o 'salir' para terminar):*"))

            except Exception as e:
                display(Markdown(f"\n❗ **Error:** {e}"))

    enviar_button.on_click(on_submit)
    display(widgets.VBox([textarea, enviar_button, output]))


In [ ]:
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

agentic_multirag(agent_executor, system_message)

Más información en:


*   https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent
*   https://langchain-ai.github.io/langgraph/reference/types/#langgraph.types.StreamMode



<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License